In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, gc
from sklearn import preprocessing
import networkx as nx
import scipy
from scipy.signal import coherence
from oct2py import Oct2Py

#Setup
base_dir = '/Users/ajunwa.c/Desktop/stress_rest_timecourse/data'
subj_list = ['sub-011','sub-012','sub-013']
out_dir = f'{base_dir}'
TR = 2.34

atlas = 'Power'
task = 'rest_run-03'

fisher_transform = False 
FDR_correct = True

def pearsonr_pval(x,y): #set up so I can extract p values from corelation matrix
        return scipy.stats.pearsonr(x,y)[1]
def ns_to_0(x): #sets non significant values in matrix to 0 based on p matrix
    locs = np.where(matrix == x)
    for loc in zip(*locs): #have to zip this since np.where returns a tuple (row indices, column indices)
        p = p_matrix_fdr.iloc[loc[0]][loc[1]]
        if p >= 0.05:
            matrix.iloc[loc[0]][loc[1]] = 0
        else:
            matrix.iloc[loc[0]][loc[1]] = x
            
    return matrix.iloc[loc[0]][loc[1]]

def coherence(x,y):
    fs = 1/TR #sampling frequency
    freq = (0.01, 0.1) #frequency band
    f, Cxy =scipy.signal.coherence(x,y,fs=fs,nperseg=len(x))
    freq_band = np.where((f>=freq[0]) & (f<= freq[1]))[0]
    Cxy = np.mean(Cxy[freq_band])
    return Cxy
    
    
    
    
            
   


In [ ]:
#create correlation matrices


for subj in subj_list:
    out_dir = f'{base_dir}/modeled/{task}/{subj}'


    #load roi timecourses
    roi_tc = pd.read_csv(f'{base_dir}/modeled/{task}/{subj}/{subj}_ROI-{atlas}_timecourse.csv')


    #create matrix with pearson correlation coefficients
    roi_tc_trunc = roi_tc.transpose()[4:]

    matrix = roi_tc_trunc.corr()

    matrix.columns = roi_tc['ROI']
    matrix.index = roi_tc['ROI']

    #fisher transform
    if fisher_transform == True:
        matrix = np.arctanh(matrix)
        matrix.replace([np.inf, -np.inf], 0, inplace=True) #change inf to 0. is this the right thing to do?
      
    
    ##correcting for multiple comparisons
    #calculate p-value matrix
    if FDR_correct == True:
        p_matrix = roi_tc_trunc.corr(method=pearsonr_pval)
        p_array = np.asarray(p_matrix)
        p_array_fdr = scipy.stats.false_discovery_control(p_array)
        p_matrix_fdr = pd.DataFrame(data = p_array_fdr,
                              index = roi_tc['ROI'],
                              columns = roi_tc['ROI'])
        #matrix = matrix.applymap(ns_to_0)
        matrix[p_matrix_fdr > .05] = 0

    

    #save
    matrix.to_csv(f'{out_dir}/{atlas}_correlation_matrix.csv')







    #visualize
    plt.imshow(matrix,cmap='bwr',vmin=-1,vmax = 1)
    plt.colorbar(label="Functional Connectivity")
    #plt.xticks(range(len(matrix)), matrix.index,fontsize=2)
    #plt.yticks(range(len(matrix)), matrix.columns,fontsize=2)
    plt.savefig(f'{out_dir}/figures/{atlas}_correlation_matrix.png')
    plt.show()

    #adjust heat map colors



In [ ]:
#create set of correlation matrices for dynamic functional connectivity 

sliding_window = 26 #TRs #60 seconds from Wang paper
overlap = 25 #TRs #2 seconds from Wang paper


for subj in subj_list:
    out_dir = f'{base_dir}/modeled/{task}/{subj}'
    
    
    #load roi timecourses
    roi_tc = pd.read_csv(f'{base_dir}/modeled/{task}/{subj}/{subj}_ROI-{atlas}_timecourse.csv')
    
    
    #create matrix with pearson correlation coefficients
    roi_tc_trunc = roi_tc.transpose()[4:]
    
    time_counter = 0
    
    window_corrs = {}
    
    for window in range(int(np.round(len(roi_tc_trunc)/(sliding_window-overlap)))):
        window_tc = roi_tc_trunc.iloc[time_counter:time_counter+sliding_window]
        if window_tc.shape[0] == sliding_window:
            time_counter += sliding_window-overlap


            matrix = window_tc.corr()



            #window_corrs[window] = matrix #adds matrix to dictionary. Alternative to saving matrices 

            #fisher transform
            if fisher_transform == True:
                matrix = np.arctanh(matrix)
                matrix.replace([np.inf, -np.inf], 0, inplace=True) #change inf to 0. is this the right thing to do?
                
            if FDR_correct == True:
                p_matrix = roi_tc_trunc.corr(method=pearsonr_pval)
                p_array = np.asarray(p_matrix)
                p_array_fdr = scipy.stats.false_discovery_control(p_array)
                p_matrix_fdr = pd.DataFrame(data = p_array_fdr,
                              index = roi_tc['ROI'],
                              columns = roi_tc['ROI'])
                #matrix = matrix.applymap(ns_to_0)
                matrix[p_matrix_fdr > .05] = 0

            #save
            if not os.path.isdir(f'{out_dir}/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR'):
                os.makedirs(f'{out_dir}/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR')

            matrix.to_csv(f'{out_dir}/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR/{atlas}_dynamic_corr_matrix_{window}_.csv')







            #visualize
            plt.imshow(matrix,cmap='coolwarm')
            plt.colorbar()

            if not os.path.isdir(f'{out_dir}/figures/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR'):
                os.makedirs(f'{out_dir}/figures/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR')

            plt.savefig(f'{out_dir}/figures/dynamic_matrices_window_{sliding_window}TR_overlap_{overlap}TR/{atlas}_corr_matrix_{window}.png')
            plt.show()
        
#look at variation within roi across windows






In [ ]:
#detecting modules 
#using networkx package

#can this be used when there are negative weights?

matrix = pd.read_csv(f'/Users/ajunwa.c/Desktop/stress_rest_timecourse/data/modeled/{task}/sub-011/{atlas}_correlation_matrix.csv')
matrix = matrix.drop('ROI', axis=1) #remove ROI column so matrix is square
matrix.index=matrix.columns #make index same as columns so function will work

graph = nx.from_pandas_adjacency(matrix) #turns matrix into networkx graph object
graph.remove_edges_from(nx.selfloop_edges(graph)) #removes self-loops (matrix diagonal)


In [ ]:
#computes modular configuration during task

louvain_partition = nx.community.louvain_communities(graph, seed=14) #with resolution of 1
len(louvain_partition) #number of partitions

#computes Q, the modularity measure

Q = nx.community.modularity(graph,louvain_partition)

#get list of ROIs with module affiliation
roi_module_affiliations = {}
for roi in roi_tc['ROI']:
    for module in louvain_partition:
        if roi in module:
            affiliation = louvain_partition.index(module)
    
    
    roi_module_affiliations[roi] = affiliation



#visualization?

In [ ]:
#compute dynamic module affiliation changes over scan

dynamic_module_affiliations = pd.DataFrame()

Qs = []

subj = 'sub-011'
task = 'rest_run-02'
roi_tc = pd.read_csv(f'{base_dir}/modeled/{task}/{subj}/{subj}_ROI-{atlas}_timecourse.csv')

for window in range(int(np.round(len(roi_tc_trunc)/(sliding_window-overlap)))-1): #-1 in this case because the last window is too small
    matrix = pd.read_csv(f'/Users/ajunwa.c/Desktop/stress_rest_timecourse/data/modeled/{task}/{subj}/dynamic_matrices_window_26TR_overlap_25TR/{atlas}_dynamic_corr_matrix_{window}_.csv')
    
    matrix = matrix.drop(matrix.columns[0], axis=1) #removes extra column so matrix is square
    
    matrix.index=roi_tc['ROI']
    matrix.columns = roi_tc['ROI']#make index same as columns so function will work
    

    graph = nx.from_pandas_adjacency(matrix) #turns matrix into networkx graph object
    graph.remove_edges_from(nx.selfloop_edges(graph)) #removes self-loops (matrix diagonal)
    
    louvain_partition = nx.community.louvain_communities(graph, seed=14) #with resolution of 1
    #print(louvain_partition)
    Q = nx.community.modularity(graph,louvain_partition)
    Qs.append(Q)
    print(Q)
    affiliations = [] #initialize list of module affiliations
    
    for roi in roi_tc['ROI']:
        for module in louvain_partition:
            #print(module)
            if roi in module:
                affiliation = louvain_partition.index(module)
                affiliations.append(affiliation)
                
                
    
    
    dynamic_module_affiliations[window] = affiliations


In [ ]:
dynamic_module_affiliations

#the algorithm may be assigning different numbers to the same modules with each iteration
#need to create visualization on brain

#columns = windows, rows = ROIs

In [ ]:
#Diagnostics

#Q_ml: Multilayer Q
#N: Number of modules
#S: Module size. Mean number of nodes in a particular module across time
#ζ: Stationarity of modules




In [ ]:
oc = Oct2Py()